Notebook ma na celu zaprezentowanie pierwszej części II. modułu projektu - czyli **obróbkę surowych danych**. Zakładamy, że dane zapisane do plików csv są poprawne i obrabiamy je do przystępnej dla oka i późniejszej kalkulacji formy.

Do zaczytania surowych plików przygotowałem sobie klasę *instrument* w skrypcie *instrument.py*. Do zaczytania potrzebujemy danych z jsona configs. Zobaczmy czego potrzebujemy z configsów i to później opiszemy.

In [3]:
import instrument
import json
import numpy as np
from tkinter.filedialog import askopenfilename

filepath = askopenfilename() # wybieramy sobie plik z dysku który chcemy zaczytać - używam tego co jest na GitHub
configs = json.loads(open('configs.json').read()) # przydatne column names, dtypes etc

for element in ['orderbook_entry_types', 'orderbook_kp', 'orderbook_sp', 'time_start', 'time_end', 'trade', 'header'  ]:
    print(element, configs[element])

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

orderbook_entry_types ['Buy', 'Sell', 'Trade']
orderbook_kp ['KpLim', 'KpIlZl', 'KpIl', 'Level']
orderbook_sp ['SpLim', 'SpIlZl', 'SpIl', 'Level']
time_start 9:00:00
time_end 16:45:00
trade ['OstTrPx', 'OstTrVol', 'Time']
header ['PxRef', 'PxHigh', 'PxLow', 'KpLim', 'KpLim_Spec', 'KpIlZl', 'KpIl', 'SpLim', 'SpLim_Spec', 'SpIlZl', 'SpIl', 'Vol', 'Turn', 'Ticks', 'OstTrPx', 'OstTrVol', 'Lop', 'Open', 'OpenTurn', 'Close', 'CloseTurn', 'Level', 'Time', 'EntryType']


Elementy z pliku configs które są używane powyżej. Nazwy kolumn zostały nadane już na poziomie I. czyli pobierania danych w Market Monitorze.
W efekcie tworzymy sobie instrument, z którego interesuje nas właściwie tylko `instr.fullbook` (jako pandas.DataFrame). Myślę, że trzeba kolumny opisać.

Chyba najlepiej zacząć od końca i opisać
**EntryType** - jest to klasyfikacja do jakiego rodzaju danych należy dany rekord. Jak domyślicie się po powyższym boxie - w obecnej  fazie interesują mnie tylko trzy typy **Buy, Sell, Trade**, dodatkowo są jeszcze **Low, High, Vol, Lop**. Oczywiście można dobrać sobie kolejne. W szczególności Lop wydaje się być kuszący, pozostałe może warto do segmentu z *feature engineering*?

## Dane wykorzystywane
### Typ Buy/Sell - czyli zmiana orderbooka
- **PxRef** - kurs odniesienia - niezmienny w ciągu dnia
- **PxHigh** - max kurs w ciągu dnia
- **PxLow** - min kurs w ciągu dnia
- **KpLim** - limit kupna (cena po której ktoś chce kupić)
- **KpLim_Spec** - specjalne limity kupna (i sprzedaży), które zasadniczo są widoczne dla nas w fazach poza sesją główną (nie wiem jak to się dokłądnie nazywa ale z boxa wyżej widać że handlujemy od 9:00 do 16:45 - kontrakty są handlowane od 8:45 do 17:05 ale te fazy sesji raczej nas tak nie interesują). Także kolumna nie użytkowana
- **KpIlZl** ilość zleceń po stronie kup
- **KpIl** ilość instrumentów do zakupu   
- Analogicznie dla strony Sp.
- **Level** - poziom orderbooka - na którym miejscu stoi dane zlecenie

### Typ Buy/Sell - czyli informacje o zawartych transakcjach
- **Ticks** - czas zawarcia transakcji - **ważne** - informacja pochodzi od brokera
- **OstTrPx** - cena zawarcia transakcji
- **OstTrVol** - wolumen zawartej transakcji

### Dodane przeze mnie
- **Time** - czas pobrania danych - **ważne** - czas mojego komputera. Należy mieć to na uwadze w przpadku opóźnień w łączu i kolejce danych do pobrania. Warto rozważyć czy nie trzeba cofnąć się do I. i nie przepatrzeć tematu.

***
## Dane niewykorzystywane
### Vol, Lop

- **Vol** - wolumen za dzień do tej pory (skumulowany)
- **Turn** - wolumen skumulowany $*$ cena po której został zawarty
- **Lop** - liczba otwartych pozycji. Właściwe tylko dla kontraktów terminowych i mówi to co jest napisane - ile kontraktów jest otwartych na całym rynku (ogółem, nie dla danego dnia)
    
### Low, High - self explaining
- **Open**
- **OpenTurn**
- **Close**
- **CloseTurn**

Poniżej przykład zaczytanego orderbooka.

In [4]:
instr = instrument.Instrument(filepath, configs)
instr.fullbook.head(10)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (0,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,PxRef,PxHigh,PxLow,KpLim,KpLim_Spec,KpIlZl,KpIl,SpLim,SpLim_Spec,SpIlZl,...,Lop,Open,OpenTurn,Close,CloseTurn,Level,Time,EntryType,DateTime,YYYYMMDD
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,637226248606960144,Trade,2020-04-16 09:07:40.696014,20200416
2,NaN,NaN,NaN,"1603,00",NaN,13.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,637226248607807884,Buy,2020-04-16 09:07:40.780788,20200416
3,NaN,NaN,NaN,"1604,00",NaN,10.0,49.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,637226248607807884,Buy,2020-04-16 09:07:40.780788,20200416
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1610,00",NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,4.0,637226248607807884,Sell,2020-04-16 09:07:40.780788,20200416
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1611,00",NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,5.0,637226248607807884,Sell,2020-04-16 09:07:40.780788,20200416
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1608,00",NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,2.0,637226248607807884,Sell,2020-04-16 09:07:40.780788,20200416
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1609,00",NaN,17.0,...,NaN,NaN,NaN,NaN,NaN,3.0,637226248607807884,Sell,2020-04-16 09:07:40.780788,20200416
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1607,00",NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,1.0,637226248607807884,Sell,2020-04-16 09:07:40.780788,20200416
9,NaN,NaN,NaN,"1602,00",NaN,11.0,41.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,637226248608097103,Buy,2020-04-16 09:07:40.809710,20200416
10,NaN,NaN,NaN,"1604,00",NaN,9.0,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,637226248608097103,Buy,2020-04-16 09:07:40.809710,20200416


Następnie używamy skryptu *data_preparation_playground.py* i istniejącej w nim funkcji *make_orderbook_files_fast* do wygenerowania potrzebnych do analizy danych. Tutaj rozbiję funkcję na czynniki pierwsze i opiszę logikę, gdyż 1) można to usprawnić (są nawet jakieś TODOsy), oraz 2) warto zajrzeć w bebechy. Najpierw funkcja zaczytuje wybrane instrumenty, czyli coś co mamy zaczytane powyżej = *instr*.

Pierwszy problem to zaadresowanie dwóch czasów - **Time** - czas pobrania lokalnego PC oraz **Ticks** czyli czas zawarcia transakcji. Potrzebujemy stworzyć 1 wektor czasu, który zachowa kolejność pobierania danych (inaczej dupa zbita i dane nie zachowają logicznej spójności). Dlatego tworzymy **NewTicks**. 
Najpierw zaznaczam sobie obszar gdzie zaszły transakcje - wiersze 70:120.

Sprawdźmy sobie teraz jak wygląda wycinek pobranych danych tam gdzie zaszła transakcja. Jak widać flaga **Ticks** istnieje tylko dla danych transakcyjnych i jest == 0 dla reszty. Niemniej - nawet dla **EntryType** == **Trade** nadaję czas PC czyli **Time**. 
Z dokumentacji API wynika, że faktycznie inne dane nie otrzymują znacznika czasu - czyli jesteśmy skazani na czas PC. Tworzy to wielki problem przy zawieszeniu pobierania ale wolałbym żeby Siata wypowiedział się czy da się to obejść.

In [5]:
take_columns = np.isin(instr.fullbook.columns,['KpLim', 'KpIlZl', 'KpIl', 'SpLim',  'SpIlZl', 'SpIl', 'Vol', 'Turn', 'Ticks', 'OstTrPx', 'OstTrVol', 'Lop', 'Open', 'OpenTurn', 'Close', 'CloseTurn', 'Level', 'Time', 'EntryType'])
instr.fullbook.loc[70:120,take_columns]

,KpLim,KpIlZl,KpIl,SpLim,SpIlZl,SpIl,Vol,Turn,Ticks,OstTrPx,OstTrVol,Lop,Open,OpenTurn,Close,CloseTurn,Level,Time,EntryType
70,NaN,NaN,NaN,"1606,00",2.0,6.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,637226248669958691,Sell
74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",5.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248670596993,Trade
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",1.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248670746576,Trade
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",5.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248670896174,Trade
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",6.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248671045774,Trade
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",1.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248671195819,Trade
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",1.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248671344978,Trade
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",1.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248671484930,Trade
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",1.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248671634216,Trade
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,637226248640000000,"1606,00",1.0,NaN,NaN,NaN,NaN,NaN,NaN,637226248671773829,Trade


In [50]:
data = instr.fullbook

ticks = data['Ticks'].values
ticks_shiff = np.roll(ticks, 1, 0) # przesuwam czas o 1
idx = np.where(np.logical_and(ticks != ticks_shiff, ticks != 0))[0]  # wyznaczam miejsca gdzie zmienia się wartość Ticks, czyli de facto wyznaczam obszary gdzie mamy dane transakcyjne - w ten sposób mamy flagę na początek i koniec (czyli dla danych zaczytanych na 74 i 110 wierszu)
idx_all = np.where(np.isin(ticks, ticks[idx]))[0] # zbieram indeksy tam gdzie były transakcje
idx_all_bool = np.isin(ticks, ticks[idx]) # i przekształcam powyższe na book (True/False)

data['NewTicks'] = data['Ticks'] # tworzymy nowy, docelowy wektor czasu
time = data['Time'].values[idx]

# TODO: test alternative for pandas.replace, which is slow
'''
idx = np.where(ticks!=0)
np.put(ticks, idx, data['Time'].values[idx])
np.put(data['NewTicks'], idx, data['Time'].values[idx])
'''

data['NewTicks'].loc[data['Ticks'] != 0] = data['Ticks'].loc[data['Ticks'] != 0].replace(ticks[idx], time)
data['NewTicks'].loc[data['Ticks'] == 0] = data['Time'].loc[data['Ticks'] == 0]

data = data.loc[:, data.columns.isin(['NewTicks','KpLim', 'KpIlZl', 'KpIl', 'SpLim', 'SpIlZl', 'SpIl', 'OstTrPx', 'OstTrVol', 'Level'])]

data.loc[70:120,:]


,KpLim,KpIlZl,KpIl,SpLim,SpIlZl,SpIl,OstTrPx,OstTrVol,Level,NewTicks
70,NaN,NaN,NaN,"1606,00",2.0,6.0,NaN,NaN,1.0,637226248669958691
74,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",5.0,NaN,637226248670596993
78,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",1.0,NaN,637226248670596993
82,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",5.0,NaN,637226248670596993
86,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",6.0,NaN,637226248670596993
90,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",1.0,NaN,637226248670596993
94,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",1.0,NaN,637226248670596993
98,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",1.0,NaN,637226248670596993
102,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",1.0,NaN,637226248670596993
106,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",1.0,NaN,637226248670596993


W całym ćwiczeniu chodziło o to żeby dane transakcyjne (tam gdzie Ticks != 0 przybrały jedną zmienną czyli 637226248670596993 - bo przecież wszystkie zaszły w tym samym czasie według danych dostarczonych przez brokera = 637226248640000000). Do innych zmiennych mamy czas PC więc tutaj żeby było spójnie to nadpisujemy czas brokera czasem PC i mamy wektor ujednoliconego czasu.

Następnie zliczamy dodajemy sobie ***CountTrades*** czyli flagę oznaczającą ile podmiotów uczestniczyło przy transakcji. Nie jest to nigdzie napisane i tak wydedukowałem - skoro mamy 10 wierszy po ***OstTrPx*** == 1606, to oznacza, że każdy przypadł na innego nabywcę bo w innym wypadku byłby jeden wiersz. Do sprawdzenia.

In [52]:
# data.to_csv('./data_new1.csv')
data_nozero = data.fillna(0)
data_nozero.groupby(by=['NewTicks', 'Level'])['OstTrVol'].sum()
#data['Level'] = data['Level'].astype(str)
grouped = data_nozero.groupby(by=['NewTicks','KpLim', 'KpIlZl', 'KpIl', 'SpLim', 'SpIlZl', 'SpIl', 'OstTrPx', 'Level'])

count_trades = data_nozero.groupby(by=['NewTicks', 'OstTrPx']).size().reset_index()
#count_trades = grouped.size().reset_index()
agg_data = grouped.sum().reset_index()
agg_data_new = agg_data.merge(count_trades, 'left',['NewTicks', 'OstTrPx'])
agg_data_new = agg_data_new.rename(columns={agg_data_new.columns[-1]:'CountTrades'})
agg_data_new = agg_data_new.fillna(0) # tutaj musiałem dodać kod względem pliku, nie wiem dokładnie czemu ale zarówno tu jak i w pliku jest taki sam efekt dopiero po dodaniu tej linii
agg_data_new['CountTrades'].loc[agg_data_new['Level'] != 0 ] = 0


agg_data_new = agg_data_new.replace(0,np.nan)
agg_data_new.loc[50:60]

,NewTicks,KpLim,KpIlZl,KpIl,SpLim,SpIlZl,SpIl,OstTrPx,Level,OstTrVol,CountTrades
50,637226248669958691,"1601,00",17.0,56.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
51,637226248669958691,"1602,00",13.0,63.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN
52,637226248669958691,"1603,00",12.0,59.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN
53,637226248669958691,"1604,00",8.0,29.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN
54,637226248669958691,"1605,00",10.0,38.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
55,637226248670596993,NaN,NaN,NaN,NaN,NaN,NaN,"1606,00",NaN,23.0,10.0
56,637226248670596993,NaN,NaN,NaN,NaN,NaN,NaN,"1607,00",NaN,1.0,1.0
57,637226248672053093,NaN,NaN,NaN,"1610,00",9.0,44.0,NaN,5.0,NaN,NaN
58,637226248672053093,"1602,00",15.0,80.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN
59,637226248672053093,"1603,00",10.0,42.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN


Następnie przygotowujemy sobie dane pod tensory to przechowywania danych. Celem jest przygotowanie obrazu orderbooka, tradebooka na każdy moment. W tym celu patrzę ile jest takich *momentów* - każdy moment to unikalny timestamp (czyli każda unikalna jednostka zapisu danych). 

Poniżej przygotownanie placeholderów pod storage orderbooka i tradebooka (odpowiednio **ob** i **tb**). Dla tradebooka mamy 2D natomiast ob jest w 3 wymiarach:
- lenght - czyli każdy timestep ma swoją macierz
- 5 - czyli ilość levelów
- 6 - czyli dane 'KpLim', 'KpIlZl', 'KpIl' oraz 'SpLim', 'SpIlZl', 'SpIl'

Dla tradebooka sprawa jest prostsza i mamy od razu 2D z 3 zmiennymi 'OstTrPx', 'OstTrVol', 'CountTrades'

In [56]:
from utils import convert_dotnet_tick

kp = agg_data_new.loc[:, ('KpLim', 'KpIlZl', 'KpIl', 'Level')].values
sp = agg_data_new.loc[:, ('SpLim', 'SpIlZl', 'SpIl', 'Level')].values
tr = agg_data_new.loc[:, ('OstTrPx', 'OstTrVol', 'CountTrades')].values
tl = agg_data_new.loc[:, 'NewTicks'].values

uniqe_timestamp_ob = np.unique(agg_data['NewTicks'].loc[agg_data['OstTrPx'] == 0])   #[:-1]

timestamps_tb = agg_data['NewTicks'].loc[agg_data['OstTrPx']!=0]
timestamps_tb_ones = np.ones((timestamps_tb.shape[0], 2), dtype=np.int64)
timestamps_tb_ones[:, 0] = timestamps_tb
timestamps_tb = timestamps_tb_ones

uniqe_timestamp_ob_zeros = np.zeros((uniqe_timestamp_ob.shape[0], 2), dtype = np.int64)
uniqe_timestamp_ob_zeros[:, 0] = uniqe_timestamp_ob
uniqe_timestamp_ob = uniqe_timestamp_ob_zeros

timestamps = np.concatenate((uniqe_timestamp_ob,timestamps_tb), axis=0)
idc = np.argsort(timestamps[:, 0])
timestamps = timestamps[idc]

length = timestamps.shape[0]
uniqe_timestamp = np.unique(timestamps)

ymd = convert_dotnet_tick(timestamps[:, 0]).astype('datetime64[D]')

ob = np.zeros([length, 5, 6], dtype=np.floating)
tb = np.zeros([length, 3], dtype=np.floating)

Dalej mamy po prostu pętlę która idzie po każdym jednym timestampie (lenght). Macierz z punktu length=1000 jest napierw równa macierzy z punktu length=999, następnie nadpisywane są tylko dane które się zmieniły (przecież dane które zapisywaliśmy to tylko są zmiany - nie ma tam stanu orderbooka, więc go odtwarzamy za pomocą tego co się zmieniło).

W ten sposób powinny powstać nam 3 pliki .npy jeden z orderbookiem, jeden z tradebookiem i jeden z wektorem czasu. Wszystkie muszą mieć taką samą długość po pierwszym wymiarze (length)